# PIMA Indian Diabetes Databse - ML Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split

## Read, Clean and Prepare data

In [2]:
data = pd.read_csv("./data/diabetes.csv")

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
data.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
# convert to a numpy array
tmp = data.to_numpy()
# separate labels from values
x_values, y_values = tmp[:,:-1], tmp[:,-1]
# Free the unnecessary space.
del data
del tmp

In [7]:
# split into training and testing set
x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.33, random_state=17)

In [8]:
print(x_train.shape)

(514, 8)


## Attempt using convoluted neural networks
I will try to solve this using neural networks, using tensorflow 

Accuracy plateau at around 68% ~ 69%, so I cancelled the training at 70% using callback.

In [9]:
import tensorflow as tf

In [10]:

class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        accuracy = 0.7 # Percentage Accuracy.
        if(logs.get('accuracy') != None) and (logs.get('accuracy') >= accuracy): # Experiment with changing this value
          print(f"\nReached {accuracy*100}% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Dense(32, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Dense(64, activation=tf.nn.leaky_relu),
    #tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [19]:
history = model.fit(
    x_train, 
    y_train, 
    steps_per_epoch=90,
    epochs=5, 
    callbacks=[callbacks]
)

Epoch 1/5
90/90 [==============================] - 0s 2ms/step - loss: 0.6121 - accuracy: 0.6822
Epoch 2/5
90/90 [==============================] - 0s 2ms/step - loss: 0.6273 - accuracy: 0.6691
Epoch 3/5
90/90 [==============================] - 0s 1ms/step - loss: 0.6175 - accuracy: 0.6933
Epoch 4/5
90/90 [==============================] - 0s 2ms/step - loss: 0.5828 - accuracy: 0.7045

Reached 70.0% accuracy so cancelling training!


In [20]:
model.evaluate(x_test, y_test)

8/8 [==============================] - 0s 2ms/step - loss: 0.5946 - accuracy: 0.7047


[0.5945795178413391, 0.7047244310379028]

## Attempt using Random Forest
Plateau at 77.6% accuracy. Better performance than CNN, but not good enough.

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [69]:
clf = RandomForestClassifier(
    max_depth=25,
    random_state=12,
    n_estimators=50, 
    bootstrap=True, 
    verbose=0
)
clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=25, n_estimators=50, random_state=12)

In [70]:
clf.score(x_test, y_test)

0.7637795275590551

## Attempt using XGB Classifier